In [2]:
import pandas as pd
sample_df = pd.read_csv("../data/medQA_openanswers.csv")

In [5]:
sample_df.head()

,Unnamed: 0,question,options,answer,reply_open
0,0,A 21-year-old sexually active male complains o...,"{'A': 'Gentamicin', 'B': 'Ciprofloxacin', 'C':...",Ceftriaxone,Penicillin.
1,1,A 5-year-old girl is brought to the emergency ...,"{'A': 'Cyclic vomiting syndrome', 'B': 'Gastro...",Cyclic vomiting syndrome,Cyclic vomiting syndrome.
2,2,A 40-year-old woman presents with difficulty f...,"{'A': 'Diazepam', 'B': 'Paroxetine', 'C': 'Zol...",Trazodone,Depression screening.
3,3,A 37-year-old female with a history of type II...,"{'A': 'Obtain an abdominal CT scan', 'B': 'Obt...",Obtain a urine analysis and urine culture,Obtain a urine culture.
4,4,A 19-year-old boy presents with confusion and ...,"{'A': 'Hypoperfusion', 'B': 'Hyperglycemia', '...",Hypoperfusion,Hyperglycemia.


In [29]:
from tech_train_functions import is_similar
sample_df.loc[:,"is correct reply"] = sample_df.apply(lambda row:row["answer"].lower() in row["reply_open"].lower(), axis=1)

In [30]:
sample_df["is correct reply"].value_counts()

is correct reply
False    26
True      4
Name: count, dtype: int64

In [6]:
from tech_train_functions import TA4HPredictor
ta4h = TA4HPredictor()

In [7]:
def is_similar_entities(gold_answers, answers):
    gold_answer_entities_list = ta4h.predict_ta4h(gold_answers)
    gold_answers_entities_short = [[entities_format(e) for e in gold_answer_entities] for gold_answer_entities in gold_answer_entities_list]
    answer_entities_list = ta4h.predict_ta4h(answers)
    answer_entities_short = [[entities_format(e) for e in answer_entities] for answer_entities in answer_entities_list]
    return compare_lists(gold_answers_entities_short, answer_entities_short)

def entities_format(e):
   return e["name"] if "name" in e else e["text"]

def compare_lists(list1, list2):
    # Initialize an empty list to store the results
    result = []

    # Iterate over each pair of lists
    for sublist1, sublist2 in zip(list1, list2):
        # Check if there is no intersection between the two sublists
        non_covered_entities = set(sublist1)-set(sublist2)
        result.append(1 if len(non_covered_entities)==0 else 0)

    return result

In [13]:
is_similar_entities(["hiv", "hbv"],["HIV","Pneumonia"])

[1, 0]

In [16]:
sample_df["is correct entities"] = is_similar_entities(sample_df["reply_open"].tolist(), sample_df["answer"].tolist())

In [20]:
sample_df["is correct entities"].value_counts()

is correct entities
0    22
1     8
Name: count, dtype: int64